In [1]:
import mlflow
import mlflow.pytorch

# absolute path
mlflow.set_tracking_uri("file:///D:/Industry/Experience/Freelance/Mariam Shahin - Red Palm/Results/mlruns")
mlflow.autolog(disable=True)

# create a clean experiment
mlflow.set_experiment("RedPalmWeeve")


<Experiment: artifact_location=('file:///D:/Industry/Experience/Freelance/Mariam Shahin - Red '
 'Palm/Results/mlruns/841599939159778227'), creation_time=1751690832749, experiment_id='841599939159778227', last_update_time=1751690832749, lifecycle_stage='active', name='RedPalmWeeve', tags={}>

In [2]:
import sys
sys.path.append("../Scripts/")
from models import RedPalmModelImage
from dataloader import RedPalmDataLoader
from torchinfo import summary
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

model = RedPalmModelImage().to("cuda")
df = pd.read_csv(r"..\Dataset\Red-Palm-Weevil-Detection-KAUST-main\audio_dataset\labels.csv")

train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
dataset_train = RedPalmDataLoader(train)
dataset_test = RedPalmDataLoader(test)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=True)

summary(model, input_size=(1, 3, 224, 224), depth=1)

c:\Users\ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                                  Output Shape              Param #
RedPalmModelImage                                       [1, 2]                    --
├─MobileNetV2: 1-1                                      [1, 1280]                 2,223,872
├─Linear: 1-2                                           [1, 512]                  655,872
├─Sequential: 1-3                                       [1, 2]                    131,842
Total params: 3,011,586
Trainable params: 3,011,586
Non-trainable params: 0
Total mult-adds (M): 300.32
Input size (MB): 0.60
Forward/backward pass size (MB): 106.86
Params size (MB): 12.05
Estimated Total Size (MB): 119.50

In [3]:
df['label'].value_counts()/len(df)

label
1    0.503291
0    0.496709
Name: count, dtype: float64

In [4]:
from tqdm.notebook import trange, tqdm

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    return test_loss, accuracy
def train(model, train_loader, optimizer, criterion, device,epoch=0):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc="Batches")):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % 10 == 0:
            mlflow.log_metric("batch_train_loss", loss, step=epoch * len(train_loader) + batch_idx)
    train_loss /= len(train_loader)
    
    return train_loss

def log_metrics(optimizer, epoch, mlflow, train_loss, test_loss, test_accuracy):
    # log metrics
    mlflow.log_metric("train_loss", train_loss, step=epoch)
    mlflow.log_metric("test_loss", test_loss, step=epoch)
    mlflow.log_metric("test_accuracy", test_accuracy, step=epoch)
    # optionally log hyperparameters
    mlflow.log_metric("learning_rate", optimizer.param_groups[0]['lr'], step = epoch)
    mlflow.log_param("batch_size", train_loader.batch_size)
    mlflow.log_param("optimizer", optimizer.__class__.__name__)
    
def check_early_stopping_sliding(test_loss, val_loss_window, patience):
    """
    Sliding-window early stopping:
    - maintains a list of last `patience` validation losses
    - if current loss improves over ALL of them, reset the window
    - else, append to the window
    - if window length equals patience and no improvement, stop
    """
    if len(val_loss_window) == 0 or test_loss < min(val_loss_window):
        # new best, reset
        val_loss_window = [test_loss]
    else:
        val_loss_window.append(test_loss)
    
    should_stop = len(val_loss_window) >= patience
    return should_stop, val_loss_window



In [ ]:
import torch
import numpy as np

labels = df['label'].values
class_counts = np.bincount(labels)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum()  # normalize


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau


best_model_path = "../Models/MinLossModel2.pt"
torch.backends.cudnn.benchmark = True
best_loss = float('inf')
class_weights = torch.tensor(class_weights, dtype=torch.float32).to("cuda")
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 50

val_loss_window = []


for epoch in trange(num_epochs, desc="Epoches"):
    train_loss = train(model, train_loader, optimizer, criterion, device, epoch=epoch)
    test_loss, test_accuracy = test(model, test_loader, criterion, device)
    
    scheduler.step(test_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    log_metrics(optimizer, epoch, mlflow, train_loss, test_loss, test_accuracy)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), best_model_path)
        mlflow.pytorch.log_model(
            pytorch_model=model,
            artifact_path="best_model_history"
        )
        print(f"✅ Saved new best model at epoch {epoch+1} with test loss {test_loss:.4f}")
    should_stop, val_loss_window = check_early_stopping_sliding(
        test_loss, val_loss_window, patience=6
    )
    if should_stop:
        print(f"🛑 Early stopping triggered at epoch {epoch+1}")
        break
model.load_state_dict(torch.load(best_model_path))
print("✅ Best model loaded from disk.")
mlflow.pytorch.log_model(model, artifact_path="final_model")


c:\Users\ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoches:   0%|          | 0/50 [00:00<?, ?it/s]

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

2025/07/05 23:09:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 1/50, Train Loss: 0.3992, Test Loss: 0.0080, Test Accuracy: 0.8936


...

Epoch 10/50, Train Loss: 0.0189, Test Loss: 0.0087, Test Accuracy: 0.9246


Batches:   0%|          | 0/65 [00:00<?, ?it/s]

C:\Users\ahmad\AppData\Local\Temp\ipykernel_10888\795462652.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))
2025/07/0

Epoch 11/50, Train Loss: 0.0271, Test Loss: 0.0054, Test Accuracy: 0.9555
🛑 Early stopping triggered at epoch 11
✅ Best model loaded from disk.


2025/07/05 23:39:53 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/07/05 23:39:58 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/07/05 23:39:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [10]:
from sklearn.metrics import f1_score, confusion_matrix

def eval(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            all_preds.extend(pred.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    
    f1 = f1_score(all_targets, all_preds, average="macro")  # or "weighted"
    cm = confusion_matrix(all_targets, all_preds)
    
    return test_loss, accuracy, f1, cm

test_loss, accuracy, f1, cm = eval(model, test_loader, criterion, device)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

F1 Score: 0.9497096764538625
Confusion Matrix:
 [[246  11]
 [ 15 245]]
